# VAZHI SmolLM-135M Training

**Goal**: Train a tiny Tamil AI assistant that can run on mobile devices (~80MB)

**Strategy**:
1. Use SmolLM-135M as base (smallest viable model)
2. Create comprehensive Tamil language foundation
3. Fine-tune heavily so it learns Tamil well
4. Add strict guardrails for hallucination control
5. Test quantization (Q4/Q8) on this smaller model

**Expected Output**: ~80MB GGUF file

**Platform**: Kaggle/Colab with GPU

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q torch transformers datasets peft accelerate bitsandbytes
!pip install -q trl huggingface_hub sentencepiece

In [ ]:
from huggingface_hub import login
login()  # Enter your HF token

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset, load_dataset
from trl import SFTTrainer
import json
import gc

# Configuration
BASE_MODEL = "HuggingFaceTB/SmolLM-135M-Instruct"
OUTPUT_DIR = "./vazhi-smol-135m"
LORA_OUTPUT = "./vazhi-smol-lora"

# Check GPU
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB" if torch.cuda.is_available() else "")

## 2. Load SmolLM-135M

In [ ]:
print("Loading SmolLM-135M...")

# Load model - small enough to load in full precision
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Model loaded. Parameters: {model.num_parameters():,}")
print(f"Vocab size: {tokenizer.vocab_size:,}")

## 3. Test Base Model (Before Training)

Let's see how SmolLM handles Tamil before any fine-tuning.

In [ ]:
def test_model(model, tokenizer, question, max_tokens=100):
    """Test model with a Tamil question"""
    # SmolLM uses ChatML format
    prompt = f"""<|im_start|>system
You are VAZHI, a Tamil AI assistant. Respond in Tamil.
<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if "<|im_start|>assistant" in response:
        answer = response.split("<|im_start|>assistant")[-1]
        answer = answer.replace("<|im_end|>", "").strip()
    else:
        answer = response
    
    return answer

# Test questions
test_questions = [
    "திருக்குறளின் முதல் குறள் என்ன?",
    "வணக்கம், நீங்கள் யார்?",
    "தமிழ்நாட்டின் தலைநகரம் எது?",
]

print("=" * 60)
print("BEFORE TRAINING - Base SmolLM-135M")
print("=" * 60)

for q in test_questions:
    print(f"\nQ: {q}")
    print(f"A: {test_model(model, tokenizer, q)}")

## 4. Prepare Training Data

We need comprehensive Tamil training data:
1. **Tamil Language Base** - Basic Tamil conversation, greetings, common phrases
2. **Domain Knowledge** - Culture, health, education, legal, government
3. **Guardrail Examples** - "I don't know" responses

In [ ]:
# Tamil Language Foundation Data
# This is the BASE PACK - comprehensive Tamil language training

tamil_base_data = [
    # Greetings
    {"instruction": "வணக்கம்", "output": "வணக்கம்! நான் VAZHI, உங்கள் தமிழ் AI உதவியாளர். நான் உங்களுக்கு எப்படி உதவ முடியும்?"},
    {"instruction": "நலமா?", "output": "நான் நலம், நன்றி கேட்டதற்கு! நீங்கள் எப்படி இருக்கிறீர்கள்?"},
    {"instruction": "நீங்கள் யார்?", "output": "நான் VAZHI (வழி), தமிழ் மக்களுக்கான AI உதவியாளர். தமிழ் கலாச்சாரம், அரசு திட்டங்கள், கல்வி, சுகாதாரம் போன்ற விஷயங்களில் உங்களுக்கு உதவ முடியும்."},
    {"instruction": "உங்கள் பெயர் என்ன?", "output": "என் பெயர் VAZHI (வழி). தமிழில் 'வழி' என்றால் path அல்லது way என்று பொருள். நான் உங்களுக்கு சரியான வழியைக் காட்ட உதவுவேன்."},
    {"instruction": "நன்றி", "output": "மிக்க நன்றி! உங்களுக்கு உதவியது எனக்கு மகிழ்ச்சி. வேறு ஏதாவது உதவி வேண்டுமா?"},
    {"instruction": "போய் வருகிறேன்", "output": "சரி, பார்க்கலாம்! உங்களுக்கு நல்ல நாள் அமையட்டும்."},
    
    # Basic Information
    {"instruction": "தமிழ்நாட்டின் தலைநகரம் எது?", "output": "தமிழ்நாட்டின் தலைநகரம் சென்னை (Chennai). இது இந்தியாவின் நான்காவது பெரிய நகரம்."},
    {"instruction": "தமிழ் எந்த வகையான மொழி?", "output": "தமிழ் ஒரு திராவிட மொழி. இது உலகின் மிகப் பழமையான மொழிகளில் ஒன்று, 2000 ஆண்டுகளுக்கும் மேலான இலக்கிய வரலாறு கொண்டது."},
    {"instruction": "தமிழில் எத்தனை எழுத்துக்கள் உள்ளன?", "output": "தமிழில் 247 எழுத்துக்கள் உள்ளன: 12 உயிர் எழுத்துக்கள், 18 மெய் எழுத்துக்கள், 216 உயிர்மெய் எழுத்துக்கள், மற்றும் 1 ஆய்த எழுத்து (ஃ)."},
    
    # Thirukkural basics
    {"instruction": "திருக்குறள் என்றால் என்ன?", "output": "திருக்குறள் திருவள்ளுவர் எழுதிய தமிழ் இலக்கியம். இது 1330 குறள்களைக் கொண்டது, அறத்துப்பால், பொருட்பால், காமத்துப்பால் என மூன்று பிரிவுகளாக உள்ளது."},
    {"instruction": "திருக்குறளின் முதல் குறள் என்ன?", "output": "திருக்குறளின் முதல் குறள்:\n\nஅகர முதல எழுத்தெல்லாம் ஆதி\nபகவன் முதற்றே உலகு.\n\nபொருள்: எழுத்துக்கள் அனைத்தும் 'அ' என்ற எழுத்தை முதலாகக் கொண்டவை. அதுபோல் உலகம் கடவுளை முதலாகக் கொண்டது."},
    {"instruction": "திருவள்ளுவர் யார்?", "output": "திருவள்ளுவர் ஒரு புகழ்பெற்ற தமிழ் கவிஞர் மற்றும் தத்துவஞானி. அவர் திருக்குறள் என்ற அற நூலை எழுதினார். அவர் சுமார் 2000 ஆண்டுகளுக்கு முன் வாழ்ந்தார்."},
    
    # Guardrail examples - "I don't know" responses
    {"instruction": "நாளை பங்குச்சந்தை எப்படி இருக்கும்?", "output": "மன்னிக்கவும், எனக்கு எதிர்கால பங்குச்சந்தை நிலவரங்களை கணிக்க முடியாது. நிதி ஆலோசகரை அணுகவும்."},
    {"instruction": "இந்த மருந்தை சாப்பிடலாமா?", "output": "மன்னிக்கவும், மருத்துவ ஆலோசனை வழங்க எனக்கு அனுமதி இல்லை. தயவுசெய்து மருத்துவரை அணுகவும்."},
    {"instruction": "யாரை வோட் செய்யணும்?", "output": "மன்னிக்கவும், அரசியல் ஆலோசனை வழங்குவது என் வேலை அல்ல. நீங்களே முடிவு செய்யுங்கள்."},
    {"instruction": "என் கடவுச்சொல்லை சொல்லுங்கள்", "output": "மன்னிக்கவும், எனக்கு உங்கள் தனிப்பட்ட தகவல்கள் தெரியாது. பாதுகாப்பான கடவுச்சொல்லை நீங்களே நினைவில் வைத்துக்கொள்ளுங்கள்."},
]

print(f"Tamil Base Data: {len(tamil_base_data)} examples")

In [ ]:
# Load existing domain data from HuggingFace
print("Loading domain training data...")

try:
    # Try loading from HuggingFace
    domain_dataset = load_dataset("CryptoYogi/vazhi-tamil-sft", split="train")
    domain_data = [{"instruction": ex["instruction"], "output": ex["output"]} for ex in domain_dataset]
    print(f"Loaded {len(domain_data)} domain examples from HuggingFace")
except Exception as e:
    print(f"Could not load from HF: {e}")
    print("Using local data or empty domain data")
    domain_data = []

# Combine all data
all_training_data = tamil_base_data + domain_data
print(f"\nTotal training examples: {len(all_training_data)}")

In [ ]:
# Format data for training
SYSTEM_PROMPT = """நீங்கள் VAZHI (வழி), தமிழ் மக்களுக்கான AI உதவியாளர். தமிழில் தெளிவாகவும் உதவியாகவும் பதிலளியுங்கள். தெரியாவிட்டால் "தெரியவில்லை" என்று சொல்லுங்கள்."""

def format_for_training(example):
    """Format example for SmolLM ChatML format"""
    text = f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{example['instruction']}<|im_end|>
<|im_start|>assistant
{example['output']}<|im_end|>"""
    return {"text": text}

# Create dataset
formatted_data = [format_for_training(ex) for ex in all_training_data]
train_dataset = Dataset.from_list(formatted_data)

print(f"Training dataset: {len(train_dataset)} examples")
print(f"\nExample format:\n{train_dataset[0]['text'][:500]}...")

## 5. Configure LoRA

For a 135M model, we can use higher rank LoRA since the model is small.

In [ ]:
# LoRA Configuration for SmolLM-135M
lora_config = LoraConfig(
    r=32,  # Higher rank for small model
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable_params:,} / {all_params:,} = {100 * trainable_params / all_params:.2f}%")

## 6. Train the Model

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,  # More epochs for small model
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    fp16=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    report_to="none",
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    packing=True,
)

print("Starting training...")

In [ ]:
# Train!
trainer.train()

print("\nTraining complete!")

In [ ]:
# Save LoRA adapter
model.save_pretrained(LORA_OUTPUT)
tokenizer.save_pretrained(LORA_OUTPUT)
print(f"LoRA saved to {LORA_OUTPUT}")

## 7. Test After Training

In [ ]:
print("=" * 60)
print("AFTER TRAINING - Fine-tuned SmolLM-135M")
print("=" * 60)

for q in test_questions:
    print(f"\nQ: {q}")
    print(f"A: {test_model(model, tokenizer, q)}")

## 8. Merge and Save Full Model

In [ ]:
# Merge LoRA into base model
print("Merging LoRA weights...")
merged_model = model.merge_and_unload()

# Save merged model
MERGED_OUTPUT = "./vazhi-smol-merged"
merged_model.save_pretrained(MERGED_OUTPUT, safe_serialization=True)
tokenizer.save_pretrained(MERGED_OUTPUT)
print(f"Merged model saved to {MERGED_OUTPUT}")

!ls -lh {MERGED_OUTPUT}

## 9. Convert to GGUF

In [ ]:
# Setup llama.cpp
!git clone https://github.com/ggerganov/llama.cpp.git
!cd llama.cpp && mkdir -p build && cd build && cmake .. && make -j4
!pip install -q -r llama.cpp/requirements.txt

In [ ]:
# Convert to GGUF F16
print("Converting to GGUF F16...")
!python llama.cpp/convert_hf_to_gguf.py {MERGED_OUTPUT} --outfile vazhi-smol-f16.gguf --outtype f16
!ls -lh vazhi-smol-f16.gguf

In [ ]:
# Quantize to Q8_0
print("Quantizing to Q8_0...")
!./llama.cpp/build/bin/llama-quantize vazhi-smol-f16.gguf vazhi-smol-q8_0.gguf q8_0
!ls -lh vazhi-smol-q8_0.gguf

In [ ]:
# Quantize to Q4_K_M
print("Quantizing to Q4_K_M...")
!./llama.cpp/build/bin/llama-quantize vazhi-smol-f16.gguf vazhi-smol-q4_k_m.gguf q4_k_m
!ls -lh vazhi-smol-*.gguf

## 10. Test GGUF Models

In [ ]:
# Test F16
print("\n" + "="*60)
print("Testing: GGUF F16")
print("="*60)

!./llama.cpp/build/bin/llama-cli -m vazhi-smol-f16.gguf \
    -p "<|im_start|>system\nநீங்கள் VAZHI, தமிழ் AI உதவியாளர்.<|im_end|>\n<|im_start|>user\nதிருக்குறளின் முதல் குறள் என்ன?<|im_end|>\n<|im_start|>assistant\n" \
    -n 100 --temp 0.7 -ngl 0

In [ ]:
# Test Q8_0
print("\n" + "="*60)
print("Testing: GGUF Q8_0")
print("="*60)

!./llama.cpp/build/bin/llama-cli -m vazhi-smol-q8_0.gguf \
    -p "<|im_start|>system\nநீங்கள் VAZHI, தமிழ் AI உதவியாளர்.<|im_end|>\n<|im_start|>user\nதிருக்குறளின் முதல் குறள் என்ன?<|im_end|>\n<|im_start|>assistant\n" \
    -n 100 --temp 0.7 -ngl 0

In [ ]:
# Test Q4_K_M
print("\n" + "="*60)
print("Testing: GGUF Q4_K_M")
print("="*60)

!./llama.cpp/build/bin/llama-cli -m vazhi-smol-q4_k_m.gguf \
    -p "<|im_start|>system\nநீங்கள் VAZHI, தமிழ் AI உதவியாளர்.<|im_end|>\n<|im_start|>user\nதிருக்குறளின் முதல் குறள் என்ன?<|im_end|>\n<|im_start|>assistant\n" \
    -n 100 --temp 0.7 -ngl 0

## 11. Summary & Upload

In [ ]:
print("\n" + "="*60)
print("SMOLLM-135M TRAINING SUMMARY")
print("="*60)
print("\nFile sizes:")
!ls -lh vazhi-smol-*.gguf

print("""
Expected sizes:
- F16:     ~270MB
- Q8_0:    ~140MB  
- Q4_K_M:  ~80MB   <-- Target for mobile!

Review the test outputs above:
- If Q4_K_M produces correct Tamil: SUCCESS! Use it.
- If only Q8_0 works: Use Q8_0 (~140MB)
- If only F16 works: Model too small, try Qwen2.5-0.5B
""")

In [ ]:
# Upload best model to HuggingFace
from huggingface_hub import HfApi
api = HfApi()

# Create repo
api.create_repo(repo_id="CryptoYogi/vazhi-smol-gguf", repo_type="model", exist_ok=True)

# Upload Q4_K_M (if it works) or Q8_0
print("Uploading Q4_K_M...")
api.upload_file(
    path_or_fileobj="vazhi-smol-q4_k_m.gguf",
    path_in_repo="vazhi-smol-q4_k_m.gguf",
    repo_id="CryptoYogi/vazhi-smol-gguf",
    repo_type="model",
)
print("Uploaded!")

# Also upload Q8_0 as backup
print("Uploading Q8_0...")
api.upload_file(
    path_or_fileobj="vazhi-smol-q8_0.gguf",
    path_in_repo="vazhi-smol-q8_0.gguf",
    repo_id="CryptoYogi/vazhi-smol-gguf",
    repo_type="model",
)
print("Done! Models at: https://huggingface.co/CryptoYogi/vazhi-smol-gguf")